In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time as time
import re
from random import randint
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import logging

In [2]:
import loaders.cityprotect as cp

Concatenate and pandaize data downloaded from CityProtect.
Given a folder, find all the files ending with "Los_Altos_Police_Department_report.csv", concatenate, and sort by date.

In [3]:
# import module
from geopy.geocoders import Nominatim
  
# initialize Nominatim API 
user_agent = 'user_me_{}'.format(randint(10000,99999))
geolocator = Nominatim(user_agent=user_agent)
 

In [4]:
dept = "Santa Clara County Office of the Sheriff"
df = cp.cityprotect("../data/raw_data/{}".format(dept.replace(" ", "_")))

Loading from 24 files.
Data file ../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff/Apr_Jun_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.
Data file ../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff/Jan_Mar_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.


In [4]:
df["inferredCity"]="unknown"

NameError: name 'df' is not defined

In [5]:
def geocode(geolocator, addr, sleep_sec):
    try:
        return geolocator.geocode(addr)
    except GeocoderTimedOut:
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        sleep(randint(1*100,sleep_sec*100)/100)
        return reverse_geocode(geolocator, latlon, sleep_sec)
    except GeocoderServiceError as e:
        logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
        logging.error(e)
        return None
    except Exception as e:
        logging.info('ERROR: Terminating due to exception {}'.format(e))
        return None

In [6]:
df = pd.read_pickle("SCCSheriff.pkl")

In [7]:
counter = 0
for index, row in df[df["inferredCity"] == "unknown"].iterrows():
    if row["inferredCity"] == "unknown":
        try:
            place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"
        except:
            print("Failed to scan: {}".format(row["blocksizedAddress"]))
            continue
        location = geocode(geolocator, place, 1)   
            
        counter = counter + 1
        if location != None:
            display_name = [s.strip() for s in location.raw["display_name"].split(",")]                         
            try:
                county_idx = display_name.index("Santa Clara County")
            except ValueError:
                print("SCC not found: {}".format(display_name))
                df.at[index, "inferredCity"] = "not_found"
                continue
            df.at[index, "inferredCity"] = display_name[county_idx-1]   
            df.at[index, "postalCode"] = display_name[-2]                            
            print("\n{} {}: {}, {}".format(counter, location.raw["display_name"], df.at[index,"inferredCity"], df.at[index, "postalCode"]))
        else:
            df.at[index, "inferredCity"] = "no resolution"
            print("NR: {} ".format(counter), end="")
        if counter/1000 == round(counter/1000):
            print("Saving...")
            df.to_pickle("SCCSheriff.pkl")
    else:
        print(".", end="")


Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan
Failed to scan: nan


ERROR:root:HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=FY+85%2C+SANTA+CLARA+COUNTY%2C+CA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


NR: 11 

NR: 12 
13 25000, Oneonta Drive, Los Altos Hills, Santa Clara County, California, 94022, United States: Los Altos Hills, 94022

14 Bubb Road, Cupertino, Santa Clara County, CAL Fire Northern Region, California, 95014, United States: Cupertino, 95014

15 9500, Burchell Road, Santa Clara County, CAL Fire Northern Region, California, 95020, United States: Burchell Road, 95020

16 13200, Uvas Road, Santa Clara County, California, 95037, United States: Uvas Road, 95037

17 12800, Stevens Court, Santa Clara County, California, 95046, United States: Stevens Court, 95046



18 2700, Ferguson Road, Santa Clara County, California, 95020, United States: Ferguson Road, 95020

19 900, Thornton Way, San Jose, Santa Clara County, California, 95128, United States: San Jose, 95128

20 Metcalf Road, San Jose, Santa Clara County, California, 95151, United States: San Jose, 95151


...

NR: 1632 
1633 Stevens Creek Boulevard, Cupertino, Santa Clara County, CAL Fire Northern Region, California, 95014, United States: Cupertino, 95014

1634 Santa Teresa Boulevard, Santa Teresa, San Jose, Santa Clara County, CAL Fire Northern Region, California, 95139, United States: San Jose, 95139

1635 JPMorgan Chase Bank, NA, 12200, Saratoga-Sunnyvale Road, Blue Hills, Saratoga, Santa Clara County, California, 95070, United States: Saratoga, 95070

1636 16000, Matilija Drive, Austin, Monte Sereno, Santa Clara County, California, 95030, United States: Monte Sereno, 95030

1637 7400, Heatherwood Drive, Cupertino, Santa Clara County, California, 95014, United States: Cupertino, 95014

1638 West Younger Avenue, Civic Center, Japantown, San Jose, Santa Clara County, California, 95110, United States: San Jose, 95110

1639 300, Almendra Avenue, Creekside Village, Los Gatos, Santa Clara County, California, 95030, United States: Los Gatos, 95030
NR: 1640 
1641 5200, Hecker Pass Highway, Santa 

ERROR:root:HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=HWY+17%2C+SANTA+CLARA+COUNTY%2C+CA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


NR: 1671 

ERROR:root:HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=HWY+17%2C+SANTA+CLARA+COUNTY%2C+CA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


NR: 1672 
1673 16100, Loretta Lane, Campbell, Santa Clara County, California, 95032, United States: Campbell, 95032

1674 14000, Loma Rio Drive, Saratoga, Santa Clara County, California, 95070, United States: Saratoga, 95070


In [8]:
df.to_pickle("SCCSheriff.pkl")

In [9]:
df["inferredCity"].value_counts()

San Jose                                              120930
Cupertino                                              51517
Saratoga                                               30051
no resolution                                          20402
Santa Clara                                             9395
                                                       ...  
Arroyo Hondo Road                                          1
Upper Uvas Creek Open Space Preserve (closed area)         1
Shannons Drive                                             1
Sunlit Oaks Court                                          1
Silveira Lane                                              1
Name: inferredCity, Length: 574, dtype: int64

In [10]:
df[df["inferredCity" == "unknown"]]

KeyError: False

        try: 
            location = geolocator.geocode(place)
        except GeocoderTimedOut as e:
            print("Location error: {}".format(e))
            naptime = 3+randint(100)
            print("Sleeping {} seconds.".format(naptime))
            time.sleep(naptime)
            user_agent = 'user_me_{}'.format(randint(10000,99999))
            geolocator = Nominatim(user_agent=user_agent)
            location = geolocator.geocode(place)
